In [ ]:
#load the library
import numpy as np
import pandas as pd

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection
from sklearn.ensemble import VotingClassifier 
from sklearn.linear_model import LogisticRegression 
# from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.datasets import load_iris 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split

import lightgbm as lgb
import optuna
import warnings
from tqdm import tqdm
import time
from sklearn.decomposition import TruncatedSVD, PCA
import joblib
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [ ]:
#transfrom categorical data to numbers
def label_encoding(train: pd.DataFrame, test: pd.DataFrame, encode_cols):
    n_train = len(train)
    train = pd.concat([train, test], sort=False).reset_index(drop=True)
    for f in encode_cols:
        try:
            lbl = preprocessing.LabelEncoder()
            train[f] = lbl.fit_transform(list(train[f].values))
        except:
            print(f)
    test = train[n_train:].reset_index(drop=True)
    train = train[:n_train]
    return train, test

In [ ]:
#import data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('gender_submission.csv')

In [ ]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [ ]:
train.shape

(891, 12)

## Fill NAs

In [ ]:
sub.PassengerId.to_list() == test.PassengerId.to_list()

True

In [ ]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
## fill age
train.Age = train.Age.fillna(0)

In [ ]:
train.Cabin = train.Cabin.fillna('Unknown')

In [ ]:
train.Embarked = train.Embarked.fillna('Unknown')

In [ ]:
train.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [ ]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [ ]:
test.Age = test.Age.fillna(0)
test.Fare = test.Fare.fillna(np.mean(test.Fare))

In [ ]:
test.Cabin = test.Cabin.fillna('Unknown')

In [ ]:
test.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

## Split folds

In [ ]:
#do k fold validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
train['fold'] = -999
for fold_id, (train_index, valid_index) in enumerate(skf.split(X=train, y=train.Survived.values)):
    train.loc[valid_index, 'fold'] = fold_id

In [ ]:
train.fold.value_counts()

0    179
1    178
2    178
3    178
4    178
Name: fold, dtype: int64

## Feature Engineering

#### 1. Name Title (Change the name to title and label encoding)

In [ ]:
train['Title'] = train.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
pd.crosstab(train['Title'], train['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [ ]:
train['Title'] = train['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 
                                             'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train['Title'] = train['Title'].replace('Mlle', 'Miss')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')
    
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [ ]:
test['Title'] = test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
pd.crosstab(test['Title'], test['Sex'])

Sex,female,male
Title,,
Col,0,2
Dona,1,0
Dr,0,1
Master,0,21
Miss,78,0
Mr,0,240
Mrs,72,0
Ms,1,0
Rev,0,2


In [ ]:
test['Title'] = test['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 
                                             'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

test['Title'] = test['Title'].replace('Mlle', 'Miss')
test['Title'] = test['Title'].replace('Ms', 'Miss')
test['Title'] = test['Title'].replace('Mme', 'Mrs')

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

train['Title'] = train['Title'].map(title_mapping)
train['Title'] = train['Title'].fillna(0)
test['Title'] = test['Title'].map(title_mapping)
test['Title'] = test['Title'].fillna(0)

In [ ]:
# train = train.drop('Name', axis=1)
# test = test.drop('Name', axis=1)

In [ ]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S,1,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,4,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S,3,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,3,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S,0,1


In [ ]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Unknown,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,Unknown,S,3
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Unknown,Q,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,Unknown,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,Unknown,S,3


#### 2.  Cut the Age and Fare column to different bins 

In [ ]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,fold,Title
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,23.799293,0.523008,0.381594,32.204208,1.997755,1.728395
std,257.353842,0.486592,0.836071,17.596074,1.102743,0.806057,49.693429,1.415800,1.030039
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,223.500000,0.000000,2.000000,6.000000,0.000000,0.000000,7.910400,1.000000,1.000000
50%,446.000000,0.000000,3.000000,24.000000,0.000000,0.000000,14.454200,2.000000,1.000000
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000,3.000000,2.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,4.000000,5.000000


In [ ]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Title
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,24.044258,0.447368,0.392344,35.627188,1.741627
std,120.810458,0.841838,17.599608,0.896760,0.981429,55.840500,1.006091
min,892.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,996.250000,1.000000,9.000000,0.000000,0.000000,7.895800,1.000000
50%,1100.500000,3.000000,24.000000,0.000000,0.000000,14.454200,1.000000
75%,1204.750000,3.000000,35.750000,1.000000,0.000000,31.500000,2.000000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200,5.000000


In [ ]:
train['Fare'] = pd.qcut(train['Fare'], 10)
test['Fare'] = pd.qcut(test['Fare'], 10)

In [ ]:
train['Age'] = pd.qcut(train['Age'], 10, duplicates='drop')
test['Age'] = pd.qcut(test['Age'], 10, duplicates='drop')

In [ ]:
type(train['SibSp'][0])

numpy.int64

In [ ]:
train['SibSp'] = pd.qcut(train['SibSp'], 4, duplicates='drop')
test['SibSp'] = pd.qcut(test['SibSp'], 4, duplicates='drop')

In [ ]:
train['Fare'] = str(train['Fare'])
test['Fare'] = str(test['Fare'])

train['Age'] = str(train['Age'])
test['Age'] = str(test['Age'])

train['SibSp'] = str(train['SibSp'])
test['SibSp'] = str(test['SibSp'])

In [ ]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,"0 (20.5, 24.0]\n1 (32.5, 38.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,A/5 21171,"0 (-0.001, 7.55]\n1 (39.688, 77.95...",Unknown,S,1,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,"0 (20.5, 24.0]\n1 (32.5, 38.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,PC 17599,"0 (-0.001, 7.55]\n1 (39.688, 77.95...",C85,C,4,3
2,3,1,3,"Heikkinen, Miss. Laina",female,"0 (20.5, 24.0]\n1 (32.5, 38.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,STON/O2. 3101282,"0 (-0.001, 7.55]\n1 (39.688, 77.95...",Unknown,S,3,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,"0 (20.5, 24.0]\n1 (32.5, 38.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,113803,"0 (-0.001, 7.55]\n1 (39.688, 77.95...",C123,S,3,3
4,5,0,3,"Allen, Mr. William Henry",male,"0 (20.5, 24.0]\n1 (32.5, 38.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,373450,"0 (-0.001, 7.55]\n1 (39.688, 77.95...",Unknown,S,0,1


In [ ]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,3,"Kelly, Mr. James",male,"0 (31.9, 39.0]\n1 (39.0, 48.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,330911,"0 (7.796, 8.05]\n1 (-0.001, 7.64...",Unknown,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,"0 (31.9, 39.0]\n1 (39.0, 48.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,363272,"0 (7.796, 8.05]\n1 (-0.001, 7.64...",Unknown,S,3
2,894,2,"Myles, Mr. Thomas Francis",male,"0 (31.9, 39.0]\n1 (39.0, 48.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,240276,"0 (7.796, 8.05]\n1 (-0.001, 7.64...",Unknown,Q,1
3,895,3,"Wirz, Mr. Albert",male,"0 (31.9, 39.0]\n1 (39.0, 48.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,315154,"0 (7.796, 8.05]\n1 (-0.001, 7.64...",Unknown,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,"0 (31.9, 39.0]\n1 (39.0, 48.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",1,3101298,"0 (7.796, 8.05]\n1 (-0.001, 7.64...",Unknown,S,3


## Label Encoding

In [ ]:
# train.Name = train.Name.str.extract('([A-Za-z]+)\.', expand=False)
# test.Name = test.Name.str.extract('([A-Za-z]+)\.', expand=False)

In [ ]:
train = train.drop('Name', axis=1)
test = test.drop('Name', axis=1)

In [ ]:
train = train.drop('PassengerId', axis=1)
test = test.drop('PassengerId', axis=1)

In [ ]:
train, test = label_encoding(train, test, ['Sex', 'Ticket', 'Cabin', 'Embarked', 'Age', 'Fare', 'SibSp'] )

# Building Model

## LGB

In [ ]:
def optuna_lgb(n_trials=100):
    
    def objective(trial):

        params = {
            "objective": "binary",
            "metric": "binary_logloss",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-5, 10, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-5, 10, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 2, 62),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 0.9),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 0.9),
            "bagging_freq": trial.suggest_int("bagging_freq", 2, 9),
            "min_child_samples": trial.suggest_int("min_child_samples", 33, 93),
            "max_depth": trial.suggest_int("max_depth", 2, 7)
        }

        # initialize oof 
        oof_train = np.zeros((len(train),))

        for i in range(5):
            train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
            train_y = train.query(f'fold!={i}').Survived

            valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
            valid_y = train.query(f'fold=={i}').Survived
            
            lgb_train = lgb.Dataset(train_x,
                                    train_y)

            lgb_eval = lgb.Dataset(valid_x,
                                   valid_y,
                                   reference=lgb_train)

            gbm = lgb.train(params,
                            lgb_train,
                            valid_sets=[lgb_train, lgb_eval],
                            num_boost_round=5000,
                            verbose_eval=-1, 
                            early_stopping_rounds=1000
                            )
            oof_preds = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
            oof_train[valid_x.index] = gbm.predict(valid_x,
                                                   num_iteration=gbm.best_iteration)
            
            
        return accuracy_score(train.Survived, oof_train.round())
    
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    best_params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
    }
    best_params.update(study.best_params)
    ## save trained model
    # initialize oof 
    oof_train = np.zeros((len(train),))
    print('*'*50)
    print('Exporting best models')
    print('*' * 50)
    # split folds using RSGKF
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived

        lgb_train = lgb.Dataset(train_x,
                                train_y)

        lgb_eval = lgb.Dataset(valid_x,
                               valid_y,
                               reference=lgb_train)

        gbm = lgb.train(best_params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_eval],
                        num_boost_round=50000,
                        verbose_eval=-1, 
                        early_stopping_rounds=1001
                        )

        oof_preds = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
        oof_train[valid_x.index] = gbm.predict(valid_x, num_iteration=gbm.best_iteration)

        joblib.dump(gbm, f'lgb_fold{i}.pkl')

In [ ]:
optuna_lgb() #0.8338945005611672(5 fold)  0.8451178451178452(10 fold)

[I 2021-10-01 03:34:46,327] A new study created in memory with name: no-name-760dc792-53f1-4679-a06c-213c301435b6


Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[2629]	training's binary_logloss: 0.232892	valid_1's binary_logloss: 0.334688
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[43]	training's binary_logloss: 0.401705	valid_1's binary_logloss: 0.451821
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[243]	training's binary_logloss: 0.356096	valid_1's binary_logloss: 0.433006
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[310]	training's binary_logloss: 0.352638	valid_1's binary_logloss: 0.358341
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[32]	training's binary_logloss: 0.406345	valid_1's binary_logloss: 0.49931
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[237]	training's binary_loglo

[I 2021-10-01 03:34:49,743] Trial 0 finished with value: 0.8316498316498316 and parameters: {'lambda_l1': 0.07583832224389743, 'lambda_l2': 1.1010202921073635, 'num_leaves': 19, 'feature_fraction': 0.48845174871096025, 'bagging_fraction': 0.7814275508425299, 'bagging_freq': 9, 'min_child_samples': 51, 'max_depth': 3}. Best is trial 0 with value: 0.8316498316498316.


Early stopping, best iteration is:
[56]	training's binary_logloss: 0.404747	valid_1's binary_logloss: 0.397434
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[50]	training's binary_logloss: 0.399654	valid_1's binary_logloss: 0.440385
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[2382]	training's binary_logloss: 0.234813	valid_1's binary_logloss: 0.330732
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[37]	training's binary_logloss: 0.414335	valid_1's binary_logloss: 0.4424
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[294]	training's binary_logloss: 0.351046	valid_1's binary_logloss: 0.432209
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[2697]	training's binary_logloss: 0.235207	valid_1's binary_logloss: 0.324423
Training until v

[I 2021-10-01 03:34:53,351] Trial 1 finished with value: 0.8361391694725028 and parameters: {'lambda_l1': 5.0337878404843074e-05, 'lambda_l2': 0.11999581928332505, 'num_leaves': 44, 'feature_fraction': 0.8388919991374862, 'bagging_fraction': 0.47626257868874283, 'bagging_freq': 3, 'min_child_samples': 63, 'max_depth': 6}. Best is trial 1 with value: 0.8361391694725028.


...

Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[33]	training's binary_logloss: 0.42157	valid_1's binary_logloss: 0.437229
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[1122]	training's binary_logloss: 0.222425	valid_1's binary_logloss: 0.339327
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[24]	training's binary_logloss: 0.420443	valid_1's binary_logloss: 0.450297
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[224]	training's binary_logloss: 0.325251	valid_1's binary_logloss: 0.422413
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[299]	training's binary_logloss: 0.319558	valid_1's binary_logloss: 0.342537
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[32]	training's binary_loglos

[I 2021-10-01 03:40:49,759] Trial 98 finished with value: 0.819304152637486 and parameters: {'lambda_l1': 0.00015734610176943012, 'lambda_l2': 0.0033701473164534056, 'num_leaves': 21, 'feature_fraction': 0.6562642098718966, 'bagging_fraction': 0.4746588212651611, 'bagging_freq': 4, 'min_child_samples': 40, 'max_depth': 7}. Best is trial 21 with value: 0.8451178451178452.


Early stopping, best iteration is:
[28]	training's binary_logloss: 0.414158	valid_1's binary_logloss: 0.439783
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[835]	training's binary_logloss: 0.33203	valid_1's binary_logloss: 0.351298
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[30]	training's binary_logloss: 0.42916	valid_1's binary_logloss: 0.451266
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[490]	training's binary_logloss: 0.351613	valid_1's binary_logloss: 0.432101
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[1684]	training's binary_logloss: 0.300289	valid_1's binary_logloss: 0.323469
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[338]	training's binary_logloss: 0.359085	valid_1's binary_logloss: 0.495253
Training until v

[I 2021-10-01 03:40:52,968] Trial 99 finished with value: 0.8260381593714927 and parameters: {'lambda_l1': 0.0026739552932489087, 'lambda_l2': 5.148929468642766e-05, 'num_leaves': 36, 'feature_fraction': 0.5636682521389806, 'bagging_fraction': 0.4079028317966495, 'bagging_freq': 5, 'min_child_samples': 56, 'max_depth': 3}. Best is trial 21 with value: 0.8451178451178452.


Early stopping, best iteration is:
[330]	training's binary_logloss: 0.365576	valid_1's binary_logloss: 0.409884
Number of finished trials: 100
Best trial:
  Value: 0.8451178451178452
  Params: 
    lambda_l1: 0.0007893224969805995
    lambda_l2: 1.1010180278677042e-05
    num_leaves: 40
    feature_fraction: 0.6800942440789662
    bagging_fraction: 0.49362633751215995
    bagging_freq: 3
    min_child_samples: 55
    max_depth: 2
**************************************************
Exporting best models
**************************************************
Training until validation scores don't improve for 1001 rounds
Early stopping, best iteration is:
[4995]	training's binary_logloss: 0.227701	valid_1's binary_logloss: 0.322258
Training until validation scores don't improve for 1001 rounds
Early stopping, best iteration is:
[22]	training's binary_logloss: 0.432327	valid_1's binary_logloss: 0.446276
Training until validation scores don't improve for 1001 rounds
Early stopping, best iteratio

## XGBoost

In [ ]:
#optuna xgboost example
#https://github.com/optuna/optuna-examples/blob/main/xgboost/xgboost_integration.py
import xgboost as xgb
def objective(trial):
    param = {
            "verbosity": 0,
            "objective": "binary:logistic",  #binary:logistic
            "eval_metric": "auc",  #map
            "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived

        dtrain = xgb.DMatrix(train_x, label=train_y)
        dvalid = xgb.DMatrix(valid_x, label=valid_y)
        
        accuracy = []
        # Add a callback for pruning.
        pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
        bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
        preds = bst.predict(dvalid)
        pred_labels = np.rint(preds)
        accuracy.append(sklearn.metrics.accuracy_score(valid_y, pred_labels))
        
        
        joblib.dump(bst, f'xgboost_fold{i}.pkl')
    return np.mean(accuracy)

    
if __name__ == "__main__":
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
    )
    study.optimize(objective, n_trials=100)
    print(study.best_trial)

[I 2021-09-30 20:21:52,267] A new study created in memory with name: no-name-93155488-580d-44d7-8547-27675f204d42


[0]	validation-auc:0.84084
[1]	validation-auc:0.83696
[2]	validation-auc:0.82655
[3]	validation-auc:0.83959
[4]	validation-auc:0.83860
[5]	validation-auc:0.83597
[6]	validation-auc:0.83992
[7]	validation-auc:0.83887
[8]	validation-auc:0.83781
[9]	validation-auc:0.84552
[0]	validation-auc:0.82995
[1]	validation-auc:0.82400
[2]	validation-auc:0.82921
[3]	validation-auc:0.82734
[4]	validation-auc:0.82801
[5]	validation-auc:0.82801
[6]	validation-auc:0.82814
[7]	validation-auc:0.82761
[8]	validation-auc:0.82794
[9]	validation-auc:0.83088
[0]	validation-auc:0.81451
[1]	validation-auc:0.81658
[2]	validation-auc:0.81297
[3]	validation-auc:0.81397
[4]	validation-auc:0.81491
[5]	validation-auc:0.81517
[6]	validation-auc:0.81497
[7]	validation-auc:0.81477
[8]	validation-auc:0.81497
[9]	validation-auc:0.81370
[0]	validation-auc:0.83483
[1]	validation-auc:0.83810
[2]	validation-auc:0.83710
[3]	validation-auc:0.83643
[4]	validation-auc:0.83536
[5]	validation-auc:0.83603
[6]	validation-auc:0.83536
[

[I 2021-09-30 20:21:52,646] Trial 0 finished with value: 0.8202247191011236 and parameters: {'booster': 'dart', 'lambda': 0.0013367915623065244, 'alpha': 2.7906661951167433e-07, 'max_depth': 9, 'eta': 4.3370876491103873e-07, 'gamma': 0.00017703622685067403, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 4.710836521456899e-07, 'skip_drop': 0.0005031690251434153}. Best is trial 0 with value: 0.8202247191011236.


[0]	validation-auc:0.85534
[1]	validation-auc:0.82148
[2]	validation-auc:0.83162
[3]	validation-auc:0.83057
[4]	validation-auc:0.83188
[5]	validation-auc:0.83188
[6]	validation-auc:0.83307
[7]	validation-auc:0.83228
[8]	validation-auc:0.83215
[9]	validation-auc:0.83241
[0]	validation-auc:0.83797
[1]	validation-auc:0.83409
[2]	validation-auc:0.83850
[3]	validation-auc:0.84118
[4]	validation-auc:0.84398
[5]	validation-auc:0.84439
[6]	validation-auc:0.84439
[7]	validation-auc:0.84559
[8]	validation-auc:0.84759
[9]	validation-auc:0.84693
[0]	validation-auc:0.79933
[1]	validation-auc:0.77313
[2]	validation-auc:0.79746
[3]	validation-auc:0.79492
[4]	validation-auc:0.80000
[5]	validation-auc:0.79425
[6]	validation-auc:0.79104
[7]	validation-auc:0.78824
[8]	validation-auc:0.78663
[9]	validation-auc:0.78717
[0]	validation-auc:0.82741
[1]	validation-auc:0.80000
[2]	validation-auc:0.81417
[3]	validation-auc:0.82086
[4]	validation-auc:0.82781
[5]	validation-auc:0.82941
[6]	validation-auc:0.83275
[

[I 2021-09-30 20:21:52,798] Trial 1 finished with value: 0.7528089887640449 and parameters: {'booster': 'gblinear', 'lambda': 0.129486555819154, 'alpha': 0.0001233726195442873}. Best is trial 0 with value: 0.8202247191011236.


...

[0]	validation-auc:0.88294
[1]	validation-auc:0.88841
[2]	validation-auc:0.88814
[3]	validation-auc:0.88814
[4]	validation-auc:0.88814
[5]	validation-auc:0.88814


[I 2021-09-30 20:22:08,747] Trial 98 pruned. Trial was pruned at iteration 5.


[0]	validation-auc:0.89638
[1]	validation-auc:0.88722
[2]	validation-auc:0.88775
[3]	validation-auc:0.89802
[4]	validation-auc:0.89644
[5]	validation-auc:0.89730
[6]	validation-auc:0.89710
[7]	validation-auc:0.89704
[8]	validation-auc:0.89769
[9]	validation-auc:0.89789
[0]	validation-auc:0.86330
[1]	validation-auc:0.86865
[2]	validation-auc:0.86009
[3]	validation-auc:0.86170
[4]	validation-auc:0.87172
[5]	validation-auc:0.87099
[6]	validation-auc:0.87079
[7]	validation-auc:0.86223
[8]	validation-auc:0.87059
[9]	validation-auc:0.87052
[0]	validation-auc:0.85007
[1]	validation-auc:0.84465
[2]	validation-auc:0.84372
[3]	validation-auc:0.84906
[4]	validation-auc:0.84639
[5]	validation-auc:0.84151
[6]	validation-auc:0.85147
[7]	validation-auc:0.84886
[8]	validation-auc:0.84920
[9]	validation-auc:0.84365
[0]	validation-auc:0.84465
[1]	validation-auc:0.84866
[2]	validation-auc:0.85247
[3]	validation-auc:0.85341
[4]	validation-auc:0.85094
[5]	validation-auc:0.85281
[6]	validation-auc:0.85033
[

[I 2021-09-30 20:22:09,062] Trial 99 finished with value: 0.8033707865168539 and parameters: {'booster': 'dart', 'lambda': 0.5164604653320886, 'alpha': 0.0017294950787021782, 'max_depth': 9, 'eta': 4.04083199670831e-07, 'gamma': 0.0009201294546623743, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 1.328334552234898e-06, 'skip_drop': 0.005980043640341497}. Best is trial 17 with value: 0.8314606741573034.


FrozenTrial(number=17, values=[0.8314606741573034], datetime_start=datetime.datetime(2021, 9, 30, 20, 21, 55, 811663), datetime_complete=datetime.datetime(2021, 9, 30, 20, 21, 56, 84335), params={'booster': 'dart', 'lambda': 0.834600783886302, 'alpha': 2.169873966851449e-05, 'max_depth': 7, 'eta': 4.274622813325059e-07, 'gamma': 2.971532595113193e-06, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.9412073456170559, 'skip_drop': 0.0008143732047330547}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': LogUniformDistribution(high=1.0, low=1e-08), 'alpha': LogUniformDistribution(high=1.0, low=1e-08), 'max_depth': IntUniformDistribution(high=9, low=1, step=1), 'eta': LogUniformDistribution(high=1.0, low=1e-08), 'gamma': LogUniformDistribution(high=1.0, low=1e-08), 'grow_policy': CategoricalDistribution(choices=('depthwise', 'lossguide')), 'sample_type': CategoricalDistribution(choices=('unifo

## Catboost

In [ ]:
#optuna catboost example
#https://github.com/optuna/optuna-examples/blob/main/catboost/catboost_simple.py
import catboost as cb
def objective(trial):
    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
        
        #save the five fold accuracy
        accuracy = []
        
        gbm = cb.CatBoostClassifier(**param)
        gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
        
        #save the best model for each fold 
        joblib.dump(gbm, f'catboost_fold{i}.pkl')
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-09-30 20:22:09,080] A new study created in memory with name: no-name-abdc0106-c62b-4272-a118-bccc166fc447
Custom logger is already specified. Specify more than one logger at same time is not thread safe.[I 2021-09-30 20:22:12,724] Trial 0 finished with value: 0.797752808988764 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.03559040922816563, 'depth': 1, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 7.669816883474842}. Best is trial 0 with value: 0.797752808988764.
[I 2021-09-30 20:22:15,026] Trial 1 finished with value: 0.797752808988764 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.02364800719483722, 'depth': 2, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 5.902265997752183}. Best is trial 0 with value: 0.797752808988764.
[I 2021-09-30 20:22:18,375] Trial 2 finished with value: 0.8089887640449438 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.0752944016

[I 2021-09-30 20:23:07,631] Trial 25 finished with value: 0.8146067415730337 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.09963306465010896, 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.21319713410932345}. Best is trial 10 with value: 0.8370786516853933.
[I 2021-09-30 20:23:10,837] Trial 26 finished with value: 0.8146067415730337 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.08355134475460894, 'depth': 12, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.22337608429169242}. Best is trial 10 with value: 0.8370786516853933.
[I 2021-09-30 20:23:12,832] Trial 27 finished with value: 0.8258426966292135 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.09926369001485122, 'depth': 10, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.3969479567276699}. Best is trial 10 with value: 0.8370786516853933.
[I 2021-09-30 20:23:14,615] Trial 28 finished with value: 

[I 2021-09-30 20:24:03,898] Trial 51 finished with value: 0.8370786516853933 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.08109487753387513, 'depth': 3, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 3.9703551114195337}. Best is trial 10 with value: 0.8370786516853933.
[I 2021-09-30 20:24:06,027] Trial 52 finished with value: 0.8258426966292135 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.08163045652830726, 'depth': 3, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 3.855543903876757}. Best is trial 10 with value: 0.8370786516853933.
[I 2021-09-30 20:24:08,036] Trial 53 finished with value: 0.8146067415730337 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.08989543995290963, 'depth': 2, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 1.3108767821627918}. Best is trial 10 with value: 0.8370786516853933.
[I 2021-09-30 20:24:10,

[I 2021-09-30 20:24:54,363] Trial 76 finished with value: 0.8370786516853933 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.06748216432630996, 'depth': 4, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 2.7739046927975046}. Best is trial 10 with value: 0.8370786516853933.
[I 2021-09-30 20:24:56,416] Trial 77 finished with value: 0.8202247191011236 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.09337669667857255, 'depth': 5, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0.9981092246944816}. Best is trial 10 with value: 0.8370786516853933.
[I 2021-09-30 20:24:58,415] Trial 78 finished with value: 0.8146067415730337 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.05875031236599851, 'depth': 4, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'}. Best is trial 10 with value: 0.8370786516853933.
[I 2021-09-30 20:25:00,328] Trial 79 finished with value: 0.8146067415

Number of finished trials: 100
Best trial:
  Value: 0.8426966292134831
  Params: 
    objective: CrossEntropy
    colsample_bylevel: 0.07571850788892448
    depth: 5
    boosting_type: Plain
    bootstrap_type: MVS


## Logistic and Random Forest and SVC

In [ ]:
#Step 1. Define an objective function to be maximized.
def objective(trial):

    classifier_name = trial.suggest_categorical("classifier", ["LogReg", "SVC", "RandomForest"])
    
    # Step 2. Setup values for the hyperparameters:
    if classifier_name == 'LogReg':
        logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
        classifier_obj = linear_model.LogisticRegression(C=logreg_c)
    elif classifier_name == "SVC":
        svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
        classifier_obj = sklearn.svm.SVC(C=svc_c, gamma="auto")
    else:
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 10, 1000)
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        classifier_obj = ensemble.RandomForestClassifier(
            max_depth=rf_max_depth, n_estimators=rf_n_estimators
        )
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'log_randomf_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-09-30 22:24:12,292] A new study created in memory with name: no-name-e8600ff9-8e3a-4f01-a759-8ca8120c2e54
[I 2021-09-30 22:24:12,436] Trial 0 finished with value: 0.6460674157303371 and parameters: {'classifier': 'LogReg', 'logreg_c': 0.00010895004457502402}. Best is trial 0 with value: 0.6460674157303371.
[I 2021-09-30 22:24:16,937] Trial 1 finished with value: 0.8258426966292135 and parameters: {'classifier': 'RandomForest', 'rf_n_estimators': 629, 'rf_max_depth': 3}. Best is trial 1 with value: 0.8258426966292135.
[I 2021-09-30 22:24:17,028] Trial 2 finished with value: 0.6348314606741573 and parameters: {'classifier': 'LogReg', 'logreg_c': 5.665470597887216e-06}. Best is trial 1 with value: 0.8258426966292135.
[I 2021-09-30 22:24:20,434] Trial 3 finished with value: 0.7865168539325843 and parameters: {'classifier': 'RandomForest', 'rf_n_estimators': 422, 'rf_max_depth': 16}. Best is trial 1 with value: 0.8258426966292135.
[I 2021-09-30 22:24:23,192] Trial 4 finished with va

[I 2021-09-30 22:25:53,332] Trial 36 finished with value: 0.8146067415730337 and parameters: {'classifier': 'LogReg', 'logreg_c': 6058051633.252974}. Best is trial 21 with value: 0.8314606741573034.
[I 2021-09-30 22:25:57,277] Trial 37 finished with value: 0.8089887640449438 and parameters: {'classifier': 'RandomForest', 'rf_n_estimators': 473, 'rf_max_depth': 14}. Best is trial 21 with value: 0.8314606741573034.
[I 2021-09-30 22:26:00,220] Trial 38 finished with value: 0.8202247191011236 and parameters: {'classifier': 'RandomForest', 'rf_n_estimators': 377, 'rf_max_depth': 6}. Best is trial 21 with value: 0.8314606741573034.
[I 2021-09-30 22:26:00,361] Trial 39 finished with value: 0.8089887640449438 and parameters: {'classifier': 'LogReg', 'logreg_c': 54411.0618532476}. Best is trial 21 with value: 0.8314606741573034.
[I 2021-09-30 22:26:00,484] Trial 40 finished with value: 0.6966292134831461 and parameters: {'classifier': 'SVC', 'svc_c': 0.553385783234753}. Best is trial 21 with va

[I 2021-09-30 22:27:34,707] Trial 73 finished with value: 0.8146067415730337 and parameters: {'classifier': 'RandomForest', 'rf_n_estimators': 86, 'rf_max_depth': 3}. Best is trial 21 with value: 0.8314606741573034.
[I 2021-09-30 22:27:35,338] Trial 74 finished with value: 0.8089887640449438 and parameters: {'classifier': 'RandomForest', 'rf_n_estimators': 78, 'rf_max_depth': 4}. Best is trial 21 with value: 0.8314606741573034.
[I 2021-09-30 22:27:35,446] Trial 75 finished with value: 0.6123595505617978 and parameters: {'classifier': 'SVC', 'svc_c': 7.74652972872992e-05}. Best is trial 21 with value: 0.8314606741573034.
[I 2021-09-30 22:27:39,294] Trial 76 finished with value: 0.8258426966292135 and parameters: {'classifier': 'RandomForest', 'rf_n_estimators': 490, 'rf_max_depth': 6}. Best is trial 21 with value: 0.8314606741573034.
[I 2021-09-30 22:27:39,602] Trial 77 finished with value: 0.8089887640449438 and parameters: {'classifier': 'RandomForest', 'rf_n_estimators': 35, 'rf_max_

Number of finished trials: 100
Best trial:
  Value: 0.8314606741573034
  Params: 
    classifier: RandomForest
    rf_n_estimators: 329
    rf_max_depth: 9


## Logistic

In [ ]:
#logistic

#Step 1. Define an objective function to be maximized.
def objective(trial):

    classifier_name = 'LogReg'
    logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
    classifier_obj = linear_model.LogisticRegression(C=logreg_c)
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'logistic_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-09-30 23:03:08,496] A new study created in memory with name: no-name-e7a9a6bc-4a6f-4735-80b6-00612735ced3
[I 2021-09-30 23:03:08,656] Trial 0 finished with value: 0.8202247191011236 and parameters: {'logreg_c': 78.46429446027122}. Best is trial 0 with value: 0.8202247191011236.
[I 2021-09-30 23:03:08,729] Trial 1 finished with value: 0.6123595505617978 and parameters: {'logreg_c': 1.6667130780805082e-09}. Best is trial 0 with value: 0.8202247191011236.
[I 2021-09-30 23:03:08,819] Trial 2 finished with value: 0.6348314606741573 and parameters: {'logreg_c': 9.915237171146237e-06}. Best is trial 0 with value: 0.8202247191011236.
[I 2021-09-30 23:03:08,953] Trial 3 finished with value: 0.8089887640449438 and parameters: {'logreg_c': 52792.61616537868}. Best is trial 0 with value: 0.8202247191011236.
[I 2021-09-30 23:03:09,024] Trial 4 finished with value: 0.6123595505617978 and parameters: {'logreg_c': 2.983684518803074e-08}. Best is trial 0 with value: 0.8202247191011236.
[I 2021-

[I 2021-09-30 23:03:14,114] Trial 43 finished with value: 0.8089887640449438 and parameters: {'logreg_c': 6.040147760176253}. Best is trial 35 with value: 0.8370786516853933.
[I 2021-09-30 23:03:14,250] Trial 44 finished with value: 0.7134831460674157 and parameters: {'logreg_c': 0.00570897923519794}. Best is trial 35 with value: 0.8370786516853933.
[I 2021-09-30 23:03:14,386] Trial 45 finished with value: 0.8426966292134831 and parameters: {'logreg_c': 0.09588367919397915}. Best is trial 45 with value: 0.8426966292134831.
[I 2021-09-30 23:03:14,527] Trial 46 finished with value: 0.8314606741573034 and parameters: {'logreg_c': 0.089073570630144}. Best is trial 45 with value: 0.8426966292134831.
[I 2021-09-30 23:03:14,640] Trial 47 finished with value: 0.6460674157303371 and parameters: {'logreg_c': 0.00016251889894232496}. Best is trial 45 with value: 0.8426966292134831.
[I 2021-09-30 23:03:14,772] Trial 48 finished with value: 0.8089887640449438 and parameters: {'logreg_c': 104.549817

[I 2021-09-30 23:03:20,103] Trial 87 finished with value: 0.8089887640449438 and parameters: {'logreg_c': 4.1062451677142064}. Best is trial 45 with value: 0.8426966292134831.
[I 2021-09-30 23:03:20,222] Trial 88 finished with value: 0.6629213483146067 and parameters: {'logreg_c': 0.0007489054531502529}. Best is trial 45 with value: 0.8426966292134831.
[I 2021-09-30 23:03:20,324] Trial 89 finished with value: 0.6460674157303371 and parameters: {'logreg_c': 5.554943930214126e-05}. Best is trial 45 with value: 0.8426966292134831.
[I 2021-09-30 23:03:20,459] Trial 90 finished with value: 0.8258426966292135 and parameters: {'logreg_c': 0.045014519579986485}. Best is trial 45 with value: 0.8426966292134831.
[I 2021-09-30 23:03:20,610] Trial 91 finished with value: 0.8202247191011236 and parameters: {'logreg_c': 0.04069927139713427}. Best is trial 45 with value: 0.8426966292134831.
[I 2021-09-30 23:03:20,745] Trial 92 finished with value: 0.8202247191011236 and parameters: {'logreg_c': 1.121

Number of finished trials: 100
Best trial:
  Value: 0.8426966292134831
  Params: 
    logreg_c: 0.09588367919397915


## SVC

In [ ]:
#Step 1. Define an objective function to be maximized.
def objective(trial):

    # Step 2. Setup values for the hyperparameters:
    classifier_name = "SVC"
    svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
    classifier_obj = sklearn.svm.SVC(C=svc_c, gamma="auto")
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'svc_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-10-01 02:32:44,159] A new study created in memory with name: no-name-0a63f536-b80b-4489-bbd4-e6f449f82b17
[I 2021-10-01 02:32:44,336] Trial 0 finished with value: 0.7078651685393258 and parameters: {'svc_c': 516066892.5463997}. Best is trial 0 with value: 0.7078651685393258.
[I 2021-10-01 02:32:44,486] Trial 1 finished with value: 0.7078651685393258 and parameters: {'svc_c': 1804963756.7487767}. Best is trial 0 with value: 0.7078651685393258.
[I 2021-10-01 02:32:44,584] Trial 2 finished with value: 0.6123595505617978 and parameters: {'svc_c': 4.622917071337435e-06}. Best is trial 0 with value: 0.7078651685393258.
[I 2021-10-01 02:32:44,713] Trial 3 finished with value: 0.7359550561797753 and parameters: {'svc_c': 1.8714761228514882}. Best is trial 3 with value: 0.7359550561797753.
[I 2021-10-01 02:32:44,878] Trial 4 finished with value: 0.7078651685393258 and parameters: {'svc_c': 393734291.1989349}. Best is trial 3 with value: 0.7359550561797753.
[I 2021-10-01 02:32:44,980] Tr

[I 2021-10-01 02:32:50,464] Trial 44 finished with value: 0.7078651685393258 and parameters: {'svc_c': 75.82661647798362}. Best is trial 33 with value: 0.7471910112359551.
[I 2021-10-01 02:32:50,596] Trial 45 finished with value: 0.6123595505617978 and parameters: {'svc_c': 0.03582274386627328}. Best is trial 33 with value: 0.7471910112359551.
[I 2021-10-01 02:32:50,738] Trial 46 finished with value: 0.7359550561797753 and parameters: {'svc_c': 1.875092629479163}. Best is trial 33 with value: 0.7471910112359551.
[I 2021-10-01 02:32:50,837] Trial 47 finished with value: 0.6123595505617978 and parameters: {'svc_c': 0.0003396555359851834}. Best is trial 33 with value: 0.7471910112359551.
[I 2021-10-01 02:32:50,941] Trial 48 finished with value: 0.6123595505617978 and parameters: {'svc_c': 1.3029824910604531e-05}. Best is trial 33 with value: 0.7471910112359551.
[I 2021-10-01 02:32:51,080] Trial 49 finished with value: 0.7078651685393258 and parameters: {'svc_c': 17.341676642051105}. Best 

[I 2021-10-01 02:32:56,285] Trial 89 finished with value: 0.7471910112359551 and parameters: {'svc_c': 1.2873939999384896}. Best is trial 33 with value: 0.7471910112359551.
[I 2021-10-01 02:32:56,408] Trial 90 finished with value: 0.6629213483146067 and parameters: {'svc_c': 0.44961173852864467}. Best is trial 33 with value: 0.7471910112359551.
[I 2021-10-01 02:32:56,536] Trial 91 finished with value: 0.7303370786516854 and parameters: {'svc_c': 4.420735578814107}. Best is trial 33 with value: 0.7471910112359551.
[I 2021-10-01 02:32:56,663] Trial 92 finished with value: 0.7415730337078652 and parameters: {'svc_c': 1.2537686328454736}. Best is trial 33 with value: 0.7471910112359551.
[I 2021-10-01 02:32:56,778] Trial 93 finished with value: 0.7471910112359551 and parameters: {'svc_c': 0.9262615146850094}. Best is trial 33 with value: 0.7471910112359551.
[I 2021-10-01 02:32:56,928] Trial 94 finished with value: 0.7078651685393258 and parameters: {'svc_c': 14.706084335508269}. Best is tri

Number of finished trials: 100
Best trial:
  Value: 0.7471910112359551
  Params: 
    svc_c: 1.4278718977518037


## RandomForest

In [ ]:
#Step 1. Define an objective function to be maximized.
def objective(trial):

    classifier_name = 'RandomForest'
    rf_n_estimators = trial.suggest_int("rf_n_estimators", 10, 1000)
    rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
#     rf_min_depth = trial.suggest_int("rf_min_depth", 2, 32, log=True)
    classifier_obj = ensemble.RandomForestClassifier(
        max_depth=rf_max_depth, n_estimators=rf_n_estimators
    )
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'rf_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-10-01 02:32:57,610] A new study created in memory with name: no-name-2b5b0160-6fc3-43e8-b228-ab4b748a87fd
[I 2021-10-01 02:32:59,958] Trial 0 finished with value: 0.8033707865168539 and parameters: {'rf_n_estimators': 329, 'rf_max_depth': 2}. Best is trial 0 with value: 0.8033707865168539.
[I 2021-10-01 02:33:06,941] Trial 1 finished with value: 0.7865168539325843 and parameters: {'rf_n_estimators': 855, 'rf_max_depth': 20}. Best is trial 0 with value: 0.8033707865168539.
[I 2021-10-01 02:33:10,474] Trial 2 finished with value: 0.7921348314606742 and parameters: {'rf_n_estimators': 430, 'rf_max_depth': 25}. Best is trial 0 with value: 0.8033707865168539.
[I 2021-10-01 02:33:14,937] Trial 3 finished with value: 0.8089887640449438 and parameters: {'rf_n_estimators': 582, 'rf_max_depth': 10}. Best is trial 3 with value: 0.8089887640449438.
[I 2021-10-01 02:33:18,397] Trial 4 finished with value: 0.8146067415730337 and parameters: {'rf_n_estimators': 468, 'rf_max_depth': 3}. Best i

[I 2021-10-01 02:35:41,969] Trial 41 finished with value: 0.8314606741573034 and parameters: {'rf_n_estimators': 267, 'rf_max_depth': 8}. Best is trial 13 with value: 0.8314606741573034.
[I 2021-10-01 02:35:44,372] Trial 42 finished with value: 0.8089887640449438 and parameters: {'rf_n_estimators': 290, 'rf_max_depth': 8}. Best is trial 13 with value: 0.8314606741573034.
[I 2021-10-01 02:35:46,606] Trial 43 finished with value: 0.8202247191011236 and parameters: {'rf_n_estimators': 276, 'rf_max_depth': 8}. Best is trial 13 with value: 0.8314606741573034.
[I 2021-10-01 02:35:48,256] Trial 44 finished with value: 0.8146067415730337 and parameters: {'rf_n_estimators': 220, 'rf_max_depth': 4}. Best is trial 13 with value: 0.8314606741573034.
[I 2021-10-01 02:35:48,939] Trial 45 finished with value: 0.8314606741573034 and parameters: {'rf_n_estimators': 83, 'rf_max_depth': 5}. Best is trial 13 with value: 0.8314606741573034.
[I 2021-10-01 02:35:49,543] Trial 46 finished with value: 0.814606

[I 2021-10-01 02:37:24,790] Trial 82 finished with value: 0.8258426966292135 and parameters: {'rf_n_estimators': 326, 'rf_max_depth': 5}. Best is trial 13 with value: 0.8314606741573034.
[I 2021-10-01 02:37:27,989] Trial 83 finished with value: 0.8146067415730337 and parameters: {'rf_n_estimators': 441, 'rf_max_depth': 4}. Best is trial 13 with value: 0.8314606741573034.
[I 2021-10-01 02:37:30,578] Trial 84 finished with value: 0.8314606741573034 and parameters: {'rf_n_estimators': 361, 'rf_max_depth': 5}. Best is trial 13 with value: 0.8314606741573034.
[I 2021-10-01 02:37:31,812] Trial 85 finished with value: 0.8258426966292135 and parameters: {'rf_n_estimators': 169, 'rf_max_depth': 5}. Best is trial 13 with value: 0.8314606741573034.
[I 2021-10-01 02:37:33,538] Trial 86 finished with value: 0.8089887640449438 and parameters: {'rf_n_estimators': 224, 'rf_max_depth': 4}. Best is trial 13 with value: 0.8314606741573034.
[I 2021-10-01 02:37:37,745] Trial 87 finished with value: 0.82022

Number of finished trials: 100
Best trial:
  Value: 0.8314606741573034
  Params: 
    rf_n_estimators: 213
    rf_max_depth: 6


## Decision Tree

In [ ]:
#Step 1. Define an objective function to be maximized.
def objective(trial):
    max_depth = trial.suggest_int("max_depth", 2, 612)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 612)
    max_leaf_nodes = int(trial.suggest_int("max_leaf_nodes", 2, 612))
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    DTC = DecisionTreeClassifier(min_samples_split = min_samples_split, 
                                 max_leaf_nodes = max_leaf_nodes,
                                 criterion = criterion)
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        DTC.fit(train_x, train_y)
        preds = DTC.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(DTC, f'dt_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-10-01 02:38:34,923] A new study created in memory with name: no-name-7c6ce501-7b5e-4bd9-a267-7cfb2a3659c2
[I 2021-10-01 02:38:34,995] Trial 0 finished with value: 0.7865168539325843 and parameters: {'max_depth': 97, 'min_samples_split': 265, 'max_leaf_nodes': 111, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7865168539325843.
[I 2021-10-01 02:38:35,067] Trial 1 finished with value: 0.8202247191011236 and parameters: {'max_depth': 75, 'min_samples_split': 27, 'max_leaf_nodes': 140, 'criterion': 'gini'}. Best is trial 1 with value: 0.8202247191011236.
[I 2021-10-01 02:38:35,126] Trial 2 finished with value: 0.8033707865168539 and parameters: {'max_depth': 540, 'min_samples_split': 494, 'max_leaf_nodes': 303, 'criterion': 'entropy'}. Best is trial 1 with value: 0.8202247191011236.
[I 2021-10-01 02:38:35,190] Trial 3 finished with value: 0.8202247191011236 and parameters: {'max_depth': 375, 'min_samples_split': 99, 'max_leaf_nodes': 64, 'criterion': 'entropy'}. Best is tr

[I 2021-10-01 02:38:37,205] Trial 33 finished with value: 0.8202247191011236 and parameters: {'max_depth': 332, 'min_samples_split': 116, 'max_leaf_nodes': 111, 'criterion': 'entropy'}. Best is trial 6 with value: 0.8258426966292135.
[I 2021-10-01 02:38:37,276] Trial 34 finished with value: 0.8146067415730337 and parameters: {'max_depth': 341, 'min_samples_split': 50, 'max_leaf_nodes': 87, 'criterion': 'entropy'}. Best is trial 6 with value: 0.8258426966292135.
[I 2021-10-01 02:38:37,344] Trial 35 finished with value: 0.8202247191011236 and parameters: {'max_depth': 406, 'min_samples_split': 79, 'max_leaf_nodes': 225, 'criterion': 'gini'}. Best is trial 6 with value: 0.8258426966292135.
[I 2021-10-01 02:38:37,409] Trial 36 finished with value: 0.8146067415730337 and parameters: {'max_depth': 563, 'min_samples_split': 33, 'max_leaf_nodes': 343, 'criterion': 'gini'}. Best is trial 6 with value: 0.8258426966292135.
[I 2021-10-01 02:38:37,474] Trial 37 finished with value: 0.78651685393258

[I 2021-10-01 02:38:39,442] Trial 67 finished with value: 0.7921348314606742 and parameters: {'max_depth': 356, 'min_samples_split': 2, 'max_leaf_nodes': 309, 'criterion': 'entropy'}. Best is trial 6 with value: 0.8258426966292135.
[I 2021-10-01 02:38:39,509] Trial 68 finished with value: 0.8202247191011236 and parameters: {'max_depth': 455, 'min_samples_split': 41, 'max_leaf_nodes': 425, 'criterion': 'entropy'}. Best is trial 6 with value: 0.8258426966292135.
[I 2021-10-01 02:38:39,576] Trial 69 finished with value: 0.8258426966292135 and parameters: {'max_depth': 430, 'min_samples_split': 80, 'max_leaf_nodes': 345, 'criterion': 'entropy'}. Best is trial 6 with value: 0.8258426966292135.
[I 2021-10-01 02:38:39,641] Trial 70 finished with value: 0.7865168539325843 and parameters: {'max_depth': 384, 'min_samples_split': 242, 'max_leaf_nodes': 482, 'criterion': 'entropy'}. Best is trial 6 with value: 0.8258426966292135.
[I 2021-10-01 02:38:39,709] Trial 71 finished with value: 0.82584269

Number of finished trials: 100
Best trial:
  Value: 0.8258426966292135
  Params: 
    max_depth: 489
    min_samples_split: 82
    max_leaf_nodes: 278
    criterion: entropy


## Ensemble Model

In [ ]:
X_train = train.drop(['Survived', 'fold'], axis=1).values
y_train = train[['Survived']].values

In [ ]:
test2 = test.drop(['Survived', 'fold'], axis=1).values

In [ ]:
#get the mean of best fold in lgb   0.8338945005611672  0.77990 selected
lgb_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./lgb_fold{i}.pkl')
    lgb_pred += model.predict(test2) / 5
    
lgb_pred = lgb_pred.round().astype(int)

In [ ]:
#get the mean of best fold in xgboost  0.8314606741573034  0.78468 selected
from xgboost import XGBClassifier
xgboost_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./xgboost_fold{i}.pkl')
    model = XGBClassifier()
    model.fit(X_train, y_train)
    xgboost_pred += model.predict_proba(test2)[:,1] / 5   #predict probability [0, 1]
    
xgboost_pred = xgboost_pred.round().astype(int)

In [ ]:
#get the mean of best fold in catboost  0.8426966292134831  nice  0.77990 selected
catboost_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./catboost_fold{i}.pkl')
    catboost_pred += model.predict(test2) / 5
    
catboost_pred = catboost_pred.round().astype(int)

In [ ]:
#get the mean of best fold in logistic  0.8426966292134831 0.78947 selected nice
logistic_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./logistic_fold{i}.pkl')
    logistic_pred += model.predict(test2) / 5
    
logistic_pred = logistic_pred.round().astype(int)

In [ ]:
#get the mean of best fold in logistic/randomForest/svc  
#randomForest win 0.8314606741573034
log_ran_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./log_randomf_fold{i}.pkl')
    log_ran_pred += model.predict(test2) / 5
    
log_ran_pred = log_ran_pred.round().astype(int)

In [ ]:
#get the mean of best fold in svc  0.7528089887640449    0.66985
svc_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./svc_fold{i}.pkl')
    svc_pred += model.predict(test2) / 5
    
svc_pred = svc_pred.round().astype(int)

In [ ]:
#get the mean of best fold in randomForest   0.8370786516853933/0.8314606741573034(add min)  0.69138/0.78468(min)
rf_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./rf_fold{i}.pkl')
    rf_pred += model.predict(test2) / 5

    
rf_pred = rf_pred.round().astype(int)

In [ ]:
#get the mean of best fold in decisionTree   0.8258426966292135   0.78947 selected nice
dt_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./dt_fold{i}.pkl')
    dt_pred += model.predict(test2) / 5

    
dt_pred = dt_pred.round().astype(int)

#### way 1:

In [ ]:
#combine lgb_pred rf_pred
# f_pro = (dt_pred + xgboost_pred)/2
f_pro = dt_pred

In [ ]:
pred = []
for i in f_pro:
    if i > 0.5:
        pred.append(1)
    else:
        pred.append(0)

#### way 2:

In [ ]:
pred2 = []
for i in range(418):
    count_0 = 0
    count_1 = 0
    for model in [lgb_pred, xgboost_pred, catboost_pred, dt_pred, logistic_pred]:
        if model[i] == 0:
            count_0 += 1
        else:
            count_1 += 1
            
    if count_0 >= count_1:
        pred2.append(0)
    else:
        pred2.append(1)
# print(pred2)   

In [ ]:
sub.Survived = pred

In [ ]:
sub

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,0
415,1307,0
416,1308,0


In [ ]:
sub.to_csv('submission_only_dt.csv', index=False)